<a href="https://colab.research.google.com/github/SherinBojappa/prompting/blob/main/ask_gpt_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 1.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 13.9 MB/s 
  Created wheel for openai: filename=openai-0.25.0-py3-none-any.whl size=55881 sha256=444decab8ab862654219a96105c5273133dc3f81e82b8f528cdb8c9dcc0d1f68
  Stored in directory: /root/.cache/pip/wheels/19/de/db/e82770b480ec30fd4a6d67108744b9c52be167c04fcf4af7b5
Successfully built openai


In [2]:
import openai

In [3]:
# open ai api key
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [4]:
# get openai api key from dashboard
f = open('open_ai_key.txt', 'r')
key = f.readlines()
f.close()
openai.api_key = key[0].strip('\n')

In [5]:
# get rte dataset from huggingface
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 8.3 MB/s 
     |████████████████████████████████| 163 kB 49.4 MB/s 
     |████████████████████████████████| 212 kB 20.1 MB/s 
     |████████████████████████████████| 95 kB 4.3 MB/s 
     |████████████████████████████████| 115 kB 51.9 MB/s 
     |████████████████████████████████| 127 kB 63.9 MB/s 
     |████████████████████████████████| 115 kB 53.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [6]:
import datasets as datasets

In [7]:
# RTE dataset from Super GLUE - https://paperswithcode.com/sota/natural-language-inference-on-rte
# 0 - enatilment
# 1 - not entailment
dataset = 'rte'

In [8]:
if dataset == 'rte':
  train_set = datasets.load_dataset('super_glue',dataset, split='train')
  dev_set = datasets.load_dataset('super_glue', dataset, split='validation')

Generating train split:   0%|          | 0/2490 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/277 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset super_glue downloaded and prepared to /root/.cache/huggingface/datasets/super_glue/rte/1.0.3/bb9675f958ebfee0d5d6dc5476fafe38c79123727a7258d515c450873dbdbbed. Subsequent calls will reuse this data.


In [9]:
import pandas as pd

In [10]:
#create a dataframe with premise, hypothesis, and labels as this will be the same everywhere.
# label = 0 entailment and label=1 non entailment
#data_df = pd.DataFrame.from_dict(new_toy_set, orient='index')
#https://towardsdatascience.com/use-the-datasets-library-of-hugging-face-in-your-next-nlp-project-94e300cca850
data_df = pd.DataFrame(dev_set)

In [15]:
def create_prompt(premise, hypothesis, template):
  prompt = "{} \n Premise: {} \n Hypothesis: {} \n Yes or No?".format(template, premise, hypothesis) 
  return prompt

In [16]:
# dataset - name of the dataset
# templates_df - df containing handwritten templates
# template_names - names of templates to pick
# given a downstream dataset and the handwritten templates and a certain template name and label category
def create_prompt_df(data_df, template, label_category):
  columns = ['template', 'prompt', 'prompt_len', 'true_label', 'label_category']
  prompt_df = pd.DataFrame(columns = columns)
  
    # create the prompt by replacing premise and hypothesis with actual premise
  # and hypothesis from the dataset.
  prompt_df['prompt'] = data_df.apply(lambda x: create_prompt(x['premise'], x['hypothesis'], template), axis=1)
  prompt_df['prompt_len'] = prompt_df.apply(lambda x: len(x['prompt']), axis=1)

  prompt_df['true_label'] = data_df['label']
  prompt_df['template'] = template
  prompt_df['label_category'] = label_category

  
  #TODO add the demonstrations as well to have few shot; for now just have
  # the zero shot prompts.
  return prompt_df

In [20]:
# this template is returned from gpt-3
template = "If a hypothesis is entailed by the premise, then it is a necessary consequence of the premise."
tgt_cat = 'yes;no'
prompt_df_yes_no_p1 = create_prompt_df(data_df, template, tgt_cat)
df = prompt_df_yes_no_p1

In [19]:
prompt_df_yes_no_p1.iloc[0]['prompt']

'If a hypothesis is entailed by the premise, then it is a necessary consequence of the premise. \n Premise: Dana Reeve, the widow of the actor Christopher Reeve, has died of lung cancer at age 44, according to the Christopher Reeve Foundation. \n Hypothesis: Christopher Reeve had an accident. \n Yes or No?'

In [24]:
prompt_df_yes_no_p1.head()

,template,prompt,prompt_len,true_label,label_category
0,"If a hypothesis is entailed by the premise, th...","If a hypothesis is entailed by the premise, th...",303,1,yes;no
1,"If a hypothesis is entailed by the premise, th...","If a hypothesis is entailed by the premise, th...",389,0,yes;no
2,"If a hypothesis is entailed by the premise, th...","If a hypothesis is entailed by the premise, th...",912,1,yes;no
3,"If a hypothesis is entailed by the premise, th...","If a hypothesis is entailed by the premise, th...",828,1,yes;no
4,"If a hypothesis is entailed by the premise, th...","If a hypothesis is entailed by the premise, th...",354,0,yes;no


In [21]:
# random sampling of the training dataset examples to create demonstrations.
# randomly sample 4 examples from the training set - pick one number and the other 3 are the consecutive

import random
num_shots = 4
max_ind = len(train_set)-4
# randint gives same number when you fix the seed
random.seed(1)
start_ind = random.randint(0, max_ind)
sample_indices = range(start_ind, start_ind + num_shots)

In [25]:
# create examples such that you can easily remove them and have a k-1 shot if they exceed the context window length
demonstration_shots = {}
for index in range(num_shots):
  ex = train_set.select(sample_indices)[index]
  prompt = create_prompt(ex['premise'], ex['hypothesis'], template)
  # append the label to the example
  target = ex['label']
  label_category = tgt_cat
  labels = label_category.rstrip().split(';')
  true_label = labels[0]    # enatailment
  false_label = labels[1]   # non enatailment
  if target == 0:
    prompt = prompt + ' ' + true_label + '\n'
  elif target == 1:
    prompt = prompt + ' ' + false_label + '\n'

  demonstration_shots[index] = prompt

In [26]:
demonstration_shots

{0: 'If a hypothesis is entailed by the premise, then it is a necessary consequence of the premise. \n Premise: In any case, the fact that this week Michael Melvill, a 63-year-old civilian pilot, guided a tiny rocket-ship more than 100 kilometres above the Earth and then glided it safely back to Earth, is a cause for celebration. \n Hypothesis: Michael Melvill guided more than 100 rocket-ships above the Earth. \n Yes or No? no\n',
 1: 'If a hypothesis is entailed by the premise, then it is a necessary consequence of the premise. \n Premise: Christopher Reeve, an actor and director who became an inspiration worldwide after being paralyzed in a horse riding accident, died Sunday of heart failure. \n Hypothesis: Christopher Reeve had an accident. \n Yes or No? yes\n',
 2: "If a hypothesis is entailed by the premise, then it is a necessary consequence of the premise. \n Premise: Further evidence emerged on Wednesday of the severe downturn in the aviation sector as Emirates pulled its giant

In [27]:
# apply demonstrations to prompts but make sure that the number of tokens for 
# each demonstration doesnt exceed 1500 based on OpenAIs recommendation
# # https://beta.openai.com/docs/introduction/key-concepts 
# One limitation to keep in mind is that your text prompt and generated 
# completion combined must be no more than the model's maximum context length 
# (for most models this is 2048 tokens, or about 1500 words).
# if the number of words increases 1500 just reduce the number of shots.
def apply_demo_to_prompts(demos, num_shots, prompt, prompt_len):
  # try num_shots first
  dem = ''
  for num in range(num_shots):
    dem = dem + demos[num]
  
  shots = num_shots
  while(len(dem) + prompt_len > 1500):
    shots = shots - 1
    dem = ''
    # empty demonstrations will be returned if the prompt length is too big.
    if shots == 0:
      break
    for num in range(shots):
      dem = dem + demos[num]

  dem = dem + prompt
  
  return dem

In [28]:
# create a column with demonstrations and prompt
df['demo_and_prompt'] = df.apply(lambda x: apply_demo_to_prompts(demonstration_shots,
                                                                                  num_shots,
                                                                                   x['prompt'], x['prompt_len']), axis=1)
df['demo_and_prompt_len'] =  df.apply(lambda x: len(x['demo_and_prompt']), axis=1)        

In [30]:
df.iloc[0]['demo_and_prompt']

'If a hypothesis is entailed by the premise, then it is a necessary consequence of the premise. \n Premise: In any case, the fact that this week Michael Melvill, a 63-year-old civilian pilot, guided a tiny rocket-ship more than 100 kilometres above the Earth and then glided it safely back to Earth, is a cause for celebration. \n Hypothesis: Michael Melvill guided more than 100 rocket-ships above the Earth. \n Yes or No? no\nIf a hypothesis is entailed by the premise, then it is a necessary consequence of the premise. \n Premise: Christopher Reeve, an actor and director who became an inspiration worldwide after being paralyzed in a horse riding accident, died Sunday of heart failure. \n Hypothesis: Christopher Reeve had an accident. \n Yes or No? yes\nIf a hypothesis is entailed by the premise, then it is a necessary consequence of the premise. \n Premise: Dana Reeve, the widow of the actor Christopher Reeve, has died of lung cancer at age 44, according to the Christopher Reeve Foundati

In [31]:
def query_gpt_3(prompt):
  # set temperature = 0 because classification problem and you want similar results all the time
  response = openai.Completion.create(engine='text-davinci-001', prompt=prompt, max_tokens = 5, temperature=0)
  return response

In [32]:
import time
import pickle

In [33]:
gpt_3_results = []
count = 1
for index, row in df.iterrows():
  print(count)
  resp = query_gpt_3(row['demo_and_prompt'])
  #print(row['demo_and_prompt'])
  gpt_3_results.append(resp)
  with open('gpt-3.pkl', 'wb') as f:
    pickle.dump(gpt_3_results, f)
  count+=1
  if count == 30:
    time.sleep(60)
    count = 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [34]:
df['GPT-3_response'] = gpt_3_results

In [47]:
df['gpt_resp'] = df['GPT-3_response'].apply(lambda x: x['choices'][0]["text"])

In [49]:
df['gpt_resp'].unique()

array([' yes', ' no', '\n\nYes. The'], dtype=object)

In [35]:
gpt_3_results[0]

<OpenAIObject text_completion id=cmpl-6A3VRjDBV5yI03yV8kV5i38aNyuIc at 0x7f48e4515dd0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " yes"
    }
  ],
  "created": 1667854777,
  "id": "cmpl-6A3VRjDBV5yI03yV8kV5i38aNyuIc",
  "model": "text-davinci-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 252,
    "total_tokens": 253
  }
}

In [36]:
# check if the label category is in the answer given by gpt
label_category = tgt_cat
labels = label_category.rstrip().split(';')
true_label = labels[0]    # entailment 0
false_label = labels[1]   # non entailment 1

In [37]:
true_label

'yes'

In [38]:
def convert_resp_to_label(resp):
  # true label is in the resp
  if true_label.lower() in resp['choices'][0]["text"].lower():
    pred_label = 0
  # false label is in the resp
  elif false_label.lower() in resp['choices'][0]["text"].lower():
    pred_label = 1
  else:
    # neither the true or false label is in the resp
    pred_label = -1

  return pred_label

In [39]:
# get the predicted label
df['pred_label'] =  df.apply(lambda x: convert_resp_to_label(x['GPT-3_response']), axis=1)

In [50]:
df['pred_label'].unique()

array([0, 1])

In [40]:
# number of matches where the response is equal to the true label
df['Match'] = df.pred_label.eq(df.true_label)

In [41]:
df.to_json("yes_no_gpt-3-prompt")

In [42]:
# count the true and false values and use that to compute the accuracy.
accuracy = df['Match'].values.sum()/((~df['Match']).values.sum() + df['Match'].values.sum())

In [43]:
print(accuracy)

0.592057761732852
